In [1]:
from PIL import Image
import torch
from torchvision import transforms, datasets,models
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import os

In [2]:
import cv2

# Windows dependencies
# - Python 2.7.6: http://www.python.org/download/
# - OpenCV: http://opencv.org/
# - Numpy -- get numpy from here because the official builds don't support x64:
#   http://www.lfd.uci.edu/~gohlke/pythonlibs/#numpy

# Mac Dependencies
# - brew install python
# - pip install numpy
# - brew tap homebrew/science
# - brew install opencv

cap = cv2.VideoCapture(0)

while(True):
    ret, frame = cap.read()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)

    cv2.imshow('frame', rgb)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        out = cv2.imwrite('capture.jpg', frame)
        break



In [3]:
VGG16 = models.vgg16(pretrained=True)
VGG16.classifier[6] = nn.Linear(4096,29)

VGG16.load_state_dict(torch.load('model_transfer.pt', map_location=lambda storage, loc: storage))

In [4]:
data = datasets.ImageFolder('asl-alphabet-test/',transform=transforms.Compose([transforms.RandomResizedCrop(224),transforms.ToTensor()]))
class_names = [item[:7].replace("_", " ") for item in data.classes]
def predict_sign(path):
    image = Image.open(path)
    in_transform = transforms.Compose([
                        transforms.CenterCrop(224),
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])])
    image = in_transform(image)[:3,:,:].unsqueeze(0)
    image = image
        
    output = VGG16(image)
    
    pred = torch.argmax(output, dim = 1)
   
    return class_names[pred]

predict_sign('capture.jpg')
    

'P'